# **MIP4-Run2**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.8 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """You are task with maximizing the water flow in a network of pipes over 10 time steps. The water enters the system at point A and exits the system at point G. Hence, you need to maximize the amount of water that flows to point G. The water enters point A according to the function f(t) = max(-4/30t^3+t^2-0.234t+3, 0), where t is the time step. There are serveral points that are connected through pipes. Each pipe has a fixed capacity. Each point has to possibility to release water out of the system in case the outgoing pipes are full. There are 12 available pipes in total but only 10 of them can be activated. Deactivated pipes can not be used to transport water. The activation status of a pipe is set before any water arrives at point A and does not change. Finally, water can not be stored at the connection points which means that any water coming in at time t needs to be flowing out at time t + 1.

These are the pipe capacities:
AB: 3
AC: 6
AF: 1
BC: 3
BD: 10
BE: 4
CE: 4
CF: 4
DE: 5
EG: 5
FB: 12
FG: 7 """

In [5]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [6]:
print(response1.choices[0].message.content)

Here is a mathematical optimization model for this problem:

Parameters:
- t: Time step, t ∈ {0,1,2,.,9,10}
- f(t): Amount of water entering point A in time step t, calculated as  max(-4/30* t^3 + t^2 - 0.234* t + 3, 0)
- Capacity of the pipes: denoted as C_ij for pipe connecting point i to point j. Values of capacities C_AB, C_AC, C_AF, .., C_FG are given in the problem.
- n: number of activated pipes (<= 10)

Decision variables:
- x_ijt: Amount of water flowing in the pipe connecting point i to point j at time t.
- y_ij: Binary variable that is 1 if the pipe from i to j is activated, 0 otherwise

Objective function:
Maximize ∑_t (x_EGt) : We want to maximize the amount of water reaching point G over all time steps.

Constraints:
1) Water conservation: For each point and each time t, the amount of water flowing into the point = the amount flowing out of it. For instance, for point B at time t, we have:
x_ABt + x_FBt = x_BCt + x_BDt + x_BEt, ∀ t

2) Capacity constraints: The flow in ea

### **Generate Pyomo Code**

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [8]:
Markdown(response2.choices[0].message.content)

Sure, Here is an example Pyomo code for this optimization problem:

```python
from pyomo.environ import *

# Initialize a model
model = ConcreteModel()

# Sets and Indexes
model.Pipes = Set(initialize=["AB", "AC","AF", "BC","BD","BE","CE","CF","DF","EF","EG","FB","FG"], doc='Pipes')
model.Time = RangeSet(0,10)

# Parameters 
model.C = Param(model.Pipes, initialize=lambda _: random.uniform(4,10), doc='Capacity')
model.f_t=Param(model.Time,initialize=lambda model, t: max(-4/30* t**3 + t**2 - 0.234* t + 3, 0), doc ="Flow_in")

# Variables 
model.x = Var(model.Pipes, model.Time, domain=NonNegativeReals)
model.y = Var(model.Pipes, domain=Binary)

# Objective 
model.z = Objective(expr = sum(model.x["EG",t] for t in model.Time), sense=maximize)

# Constraints
def flow_rule1(model, t):
    return model.f_t[t] == model.x["AB",t] + model.x["AC",t] + model.x["AF",t]
model.flowrule_A = Constraint(model.Time, rule=flow_rule1)

def flow_rule2(model, t):
    return model.x["AB",t] + model.x["FB",t] == model.x["BC",t] + model.x["BD",t] + model.x["BE",t]
model.flowrule_B = Constraint(model.Time, rule=flow_rule2)

# Add similar constraints for other points

def capc_Rule(model, pipe,t):
    return model.x[pipe,t] <= model.y[pipe] * model.C[pipe]
model.capacrule = Constraint(model.Pipes,model.Time, rule=capc_Rule)

def pipe_Rule(model):
    return sum(model.y[pipe] for pipe in model.Pipes) <= 10
model.piperule = Constraint(rule=pipe_Rule)

# to solve
solver=SolverFactory('glpk')
solver.solve(model)
```
In this problem, the capacities(C) of pipes are randomly initialized between 4 and 10. These are the sample data for this problem. Make sure you replace the constraint flow_rule2 and similar constraints with acceptable conditions according to your needs.


### **Run the code Generated by GPT4**

In [19]:
from pyomo.environ import *
import random
# Initialize a model
model = ConcreteModel()

# Sets and Indexes
model.Pipes = Set(initialize=["AB", "AC","AF", "BC","BD","BE","CE","CF","DF","EF","EG","FB","FG"], doc='Pipes')
model.Time = RangeSet(0,10)

# Parameters
model.C = Param(model.Pipes, initialize=lambda _: random.uniform(4,10), doc='Capacity')
model.f_t=Param(model.Time,initialize=lambda model, t: max(-4/30* t**3 + t**2 - 0.234* t + 3, 0), doc ="Flow_in")

# Variables
model.x = Var(model.Pipes, model.Time, domain=NonNegativeReals)
model.y = Var(model.Pipes, domain=Binary)

# Objective
model.z = Objective(expr = sum(model.x["EG",t] for t in model.Time), sense=maximize)

# Constraints
def flow_rule1(model, t):
    return model.f_t[t] == model.x["AB",t] + model.x["AC",t] + model.x["AF",t]
model.flowrule_A = Constraint(model.Time, rule=flow_rule1)

def flow_rule2(model, t):
    return model.x["AB",t] + model.x["FB",t] == model.x["BC",t] + model.x["BD",t] + model.x["BE",t]
model.flowrule_B = Constraint(model.Time, rule=flow_rule2)

# Add similar constraints for other points

def capc_Rule(model, pipe,t):
    return model.x[pipe,t] <= model.y[pipe] * model.C[pipe]
model.capacrule = Constraint(model.Pipes,model.Time, rule=capc_Rule)

def pipe_Rule(model):
    return sum(model.y[pipe] for pipe in model.Pipes) <= 10
model.piperule = Constraint(rule=pipe_Rule)

# to solve
solver=SolverFactory('glpk')
solver.solve(model)


{'Problem': [{'Name': 'unknown', 'Lower bound': 46.2344080116433, 'Upper bound': 46.2344080116433, 'Number of objectives': 1, 'Number of constraints': 166, 'Number of variables': 156, 'Number of nonzeros': 387, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '7', 'Number of created subproblems': '7'}}, 'Error rc': 0, 'Time': 0.008052349090576172}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [26]:
from pyomo.environ import *
import random
# Initialize a model
model = ConcreteModel()

# Sets and Indexes
model.Pipes = Set(initialize=["AB", "AC","AF", "BC","BD","BE","CE","CF","DE","EG","FB","FG"], doc='Pipes')
model.Time = RangeSet(0,10)

# Parameters
capacities = [3,6,1,3,10,4,4,4,5,5,12,7]
model.C = Param(model.Pipes, initialize={pipe: capacities[idx] for idx, pipe in enumerate(model.Pipes)}, doc='Capacity')
model.f_t=Param(model.Time,initialize=lambda model, t: max(-4/30* t**3 + t**2 - 0.234* t + 3, 0), doc ="Flow_in")

# Variables
model.x = Var(model.Pipes, model.Time, domain=NonNegativeReals)
model.y = Var(model.Pipes, domain=Binary)

# Objective
model.z = Objective(expr = sum(model.x["EG",t] for t in model.Time), sense=maximize)

# Constraints

def flow_rule1(model, t):
    return model.f_t[t] == model.x["AB",t] + model.x["AC",t] + model.x["AF",t]
model.flowrule_A = Constraint(model.Time, rule=flow_rule1)

def flow_rule2(model, t):
    return model.x["AB",t] + model.x["FB",t] == model.x["BC",t] + model.x["BD",t] + model.x["BE",t]
model.flowrule_B = Constraint(model.Time, rule=flow_rule2)

def flow_rule3(model, t):
    return model.x["AC",t] + model.x["BC",t] == model.x["CE",t] + model.x["CF",t]
model.flowrule_3 = Constraint(model.Time, rule=flow_rule3)

def flow_rule4(model, t):
    return model.x["BD",t] == model.x["DE",t]
model.flowrule_4 = Constraint(model.Time, rule=flow_rule4)

def flow_rule5(model, t):
    return model.x["EG",t] == model.x["CE",t] + model.x["DE",t]
model.flowrule_5 = Constraint(model.Time, rule=flow_rule5)

def flow_rule6(model, t):
    return model.x["FG",t] == model.x["CF",t] + model.x["AF",t]
model.flowrule_6 = Constraint(model.Time, rule=flow_rule6)

# Add similar constraints for other points

def capc_Rule(model, pipe,t):
    return model.x[pipe,t] <= model.y[pipe] * model.C[pipe]
model.capacrule = Constraint(model.Pipes,model.Time, rule=capc_Rule)

def pipe_Rule(model):
    return sum(model.y[pipe] for pipe in model.Pipes) <= 10
model.piperule = Constraint(rule=pipe_Rule)

# to solve
solver=SolverFactory('glpk')
solver.solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 199, 'Number of variables': 144, 'Number of nonzeros': 496, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'infeasible', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.005341291427612305}]}